Modeling main objectives:
1) maintain the huge size of the data set which will be challenging when appling diffrent modeling approch
2) appling three diffrent models and comparing there rusullts using two Evaluation Metrics
   - we are going to use model Accuracy scare range from 0 to 1 A higher score indicates a better fit. 
   - We are going to use Mean absolute error (MAE): MAE is a measure of the average absolute difference between the predicted values and the actual values.
 
 * Compare the Model Accuracy score to MAE scores for different models. The model with the lowest MAE and the highest model score we are going to select.
 * checking the emportancy of the features for the model.
3) Tune the algorithm selected the best model wiht highest evalutions from the above step using cross valdiation to select the best parameters for the model.

In [32]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, recall_score,precision_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostRegressor,AdaBoostClassifier
import xgboost as xgb
from sklearn.model_selection import cross_validate,GridSearchCV

In [2]:
# loading the dataset.
df = pd.read_csv('../data/processed/processed_data.csv')
df

,age_group,length_of_stay,type_of_admission,ccs_diagnosis_description,ccs_procedure_description,apr_drg_description,apr_mdc_description,apr_severity_of_illness_description,apr_risk_of_mortality,gender_M,apr_medical_surgical_description_Surgical,emergency_department_indicator_Y
0,3,5,7.29,11.69,6.36,12.58,11.66,1,1,0,0,1
1,1,8,7.29,11.69,6.36,12.58,11.66,2,1,0,0,1
2,2,8,7.29,11.69,6.36,12.58,11.66,1,1,1,0,1
3,3,4,7.29,5.87,6.36,4.89,7.36,1,1,0,0,1
4,3,3,7.29,6.04,6.36,5.74,7.21,2,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
698106,3,3,7.29,8.01,6.16,5.38,6.91,3,3,0,1,1
698107,1,1,7.29,4.71,7.35,4.78,5.19,3,3,0,0,1
698108,4,3,7.29,6.85,4.51,6.66,6.91,3,3,1,0,1
698109,4,2,7.29,10.68,6.20,12.65,9.68,4,4,0,1,1


In [3]:
# Geitting Information about the Data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698111 entries, 0 to 698110
Data columns (total 12 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   age_group                                  698111 non-null  int64  
 1   length_of_stay                             698111 non-null  int64  
 2   type_of_admission                          698111 non-null  float64
 3   ccs_diagnosis_description                  698111 non-null  float64
 4   ccs_procedure_description                  698111 non-null  float64
 5   apr_drg_description                        698111 non-null  float64
 6   apr_mdc_description                        698111 non-null  float64
 7   apr_severity_of_illness_description        698111 non-null  int64  
 8   apr_risk_of_mortality                      698111 non-null  int64  
 9   gender_M                                   698111 non-null  int64  
 10  apr_medi

In [4]:
# statistical description for numerical features
df.describe()

,age_group,length_of_stay,type_of_admission,ccs_diagnosis_description,ccs_procedure_description,apr_drg_description,apr_mdc_description,apr_severity_of_illness_description,apr_risk_of_mortality,gender_M,apr_medical_surgical_description_Surgical,emergency_department_indicator_Y
count,698111.000000,698111.000000,698111.000000,698111.000000,698111.000000,698111.000000,698111.000000,698111.000000,698111.000000,698111.000000,698111.000000,698111.000000
mean,2.980360,6.742923,7.193846,7.082028,7.113087,7.034424,7.127619,2.418403,2.056332,0.484483,0.274614,0.659467
std,0.956168,5.352194,0.222143,2.063644,1.925167,2.459052,1.493436,0.906452,0.985116,0.499760,0.446320,0.473889
min,1.000000,1.000000,4.760000,2.000000,1.500000,1.880000,3.930000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,7.210000,5.860000,6.090000,5.360000,6.590000,2.000000,1.000000,0.000000,0.000000,0.000000
50%,3.000000,5.000000,7.290000,6.850000,6.400000,6.660000,6.840000,2.000000,2.000000,0.000000,0.000000,1.000000
75%,4.000000,9.000000,7.290000,7.890000,8.360000,7.970000,7.360000,3.000000,3.000000,1.000000,1.000000,1.000000
max,4.000000,20.000000,7.290000,14.090000,18.640000,19.220000,11.660000,4.000000,4.000000,1.000000,1.000000,1.000000


In [5]:
#checking for duplication in data 
df.duplicated().sum()

0

## 1) Maintain the huge size of the data set which will be challenging when appling diffrent modeling approch:
##### Although the size of the data after cleaning and preprocessing decreese greatly to 143,500 entries instance and 7 features but it is still challenging to deal with it for limted pressoer laptob so we are going take small sample to run the models more efficently while maintaing the same characteristics of the data by usin straitify method.

In [6]:
# divede the data fram to x = features & y = target
x = df.drop("length_of_stay", axis = 1)
y = df["length_of_stay"]

In [7]:
# taking sample of the data using stratify to maintain data characteristics
main_x,sample_x,main_y,sample_y = train_test_split(x,y,test_size=0.03,random_state=42,stratify=y)

In [8]:
len(sample_y)

20944

In [9]:
# now we will split sample data to train and test sets 
x_train,x_test,y_train,y_test = train_test_split(sample_x,sample_y,test_size=0.3,random_state=42)

In [10]:
# Using Stander Scaler for x_train, x_test
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [11]:
# trying three different models to compare between them
models = {"lr": LinearRegression(),
          "knn": KNeighborsRegressor(),
          "rf": RandomForestRegressor(),
        "Adboost": AdaBoostRegressor()}

In [12]:
# running the three different models and check the result using for loop
for name,model in models.items():
    print("----------- ", name, " -------------")
    model.fit(x_train_scaled,y_train)
    y_train_pred = model.predict(x_train_scaled)
    y_test_pred = model.predict(x_test_scaled)
    print("Train Acurracy Score", round(model.score(x_train_scaled,y_train),2),"Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
    print("Train MAE Score: ", mean_absolute_error(y_train_pred,y_train),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
    print("*"*60)

-----------  lr  -------------
Train Acurracy Score 0.29 Test Acurracy Score 0.28
Train MAE Score:  3.445978394939736 Test MAE Score:  3.48
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.46 Test Acurracy Score 0.18
Train MAE Score:  2.9624283765347887 Test MAE Score:  3.67
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.88 Test Acurracy Score 0.22
Train MAE Score:  1.3780031406444095 Test MAE Score:  3.59
************************************************************
-----------  Adboost  -------------
Train Acurracy Score 0.23 Test Acurracy Score 0.22
Train MAE Score:  3.8351642910145674 Test MAE Score:  3.87
************************************************************


##### checking the emportancy of the features for the random forest model. by making data frame for features importancy

In [13]:
feat_imp = list(model.feature_importances_)
col_name = list(x_train.columns)
feat_imp_df = pd.DataFrame({"col_name":col_name,"feat_imp":feat_imp})

In [14]:
feat_imp_df[['col_name','feat_imp']].sort_values(by='feat_imp')

,col_name,feat_imp
0,age_group,0.000000
8,gender_M,0.000000
10,emergency_department_indicator_Y,0.000000
1,type_of_admission,0.006836
5,apr_mdc_description,0.029507
9,apr_medical_surgical_description_Surgical,0.034156
7,apr_risk_of_mortality,0.059769
3,ccs_procedure_description,0.096865
2,ccs_diagnosis_description,0.099474
4,apr_drg_description,0.256272


Dropping the apr_medical_surgical_description_Surgical as it is show least importancy and check the model for any improvement.

In [15]:
# dropping apr_medical_surgical_description_Surgical from both x_train & x_test
x_train.drop(['apr_medical_surgical_description_Surgical'],axis=1,inplace=True)
x_test.drop(['apr_medical_surgical_description_Surgical'],axis=1,inplace=True)

In [16]:
# Using Stander Scaler for x_train, x_test
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [17]:
# trying three different models to compare between them
models = {"lr": LinearRegression(),
          "knn": KNeighborsRegressor(),
          "rf": RandomForestRegressor(),
         "Adboost": AdaBoostRegressor()}

In [18]:
# running the three different models and check the result using for loop
for name,model in models.items():
    print("----------- ", name, " -------------")
    model.fit(x_train_scaled,y_train)
    y_train_pred = model.predict(x_train_scaled)
    y_test_pred = model.predict(x_test_scaled)
    print("Train Acurracy Score", round(model.score(x_train_scaled,y_train),2),"Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
    print("Train MAE Score: ", mean_absolute_error(y_train_pred,y_train),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
    print("*"*60)

-----------  lr  -------------
Train Acurracy Score 0.29 Test Acurracy Score 0.28
Train MAE Score:  3.4508678469205063 Test MAE Score:  3.48
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.45 Test Acurracy Score 0.17
Train MAE Score:  2.9948567530695773 Test MAE Score:  3.67
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.88 Test Acurracy Score 0.22
Train MAE Score:  1.3800275322657485 Test MAE Score:  3.59
************************************************************
-----------  Adboost  -------------
Train Acurracy Score 0.22 Test Acurracy Score 0.2
Train MAE Score:  3.9007970483995207 Test MAE Score:  3.96
************************************************************


* The models show No significant imporvement imporvement 

#### We are going to use polynomial Features with diffrent degree and check if there are any improvement

In [19]:
for i in [2,3,4]:
    print("----------- ", i, " -------------")
    poly2 = PolynomialFeatures(degree=i)
    x_train_poly = poly2.fit_transform(x_train)
    x_test_poly = poly2.transform(x_test)
    for name,model in models.items():
        print("----------- ", name, " -------------")
        model.fit(x_train_poly,y_train)
        y_train_pred = model.predict(x_train_poly)
        y_test_pred = model.predict(x_test_poly)
        print("Train Acurracy Score", round(model.score(x_train_poly,y_train),2), "Test Acurracy Score", round(model.score(x_test_poly,y_test),2))
        print("Train MAE Score: ", round(mean_absolute_error(y_train_pred,y_train),2),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
        print("*"*60)

-----------  2  -------------
-----------  lr  -------------
Train Acurracy Score 0.31 Test Acurracy Score 0.29
Train MAE Score:  3.38 Test MAE Score:  3.44
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.46 Test Acurracy Score 0.18
Train MAE Score:  2.96 Test MAE Score:  3.67
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.88 Test Acurracy Score 0.23
Train MAE Score:  1.38 Test MAE Score:  3.58
************************************************************
-----------  Adboost  -------------
Train Acurracy Score 0.25 Test Acurracy Score 0.23
Train MAE Score:  3.78 Test MAE Score:  3.83
************************************************************
-----------  3  -------------
-----------  lr  -------------
Train Acurracy Score 0.33 Test Acurracy Score 0.29
Train MAE Score:  3.32 Test MAE Score:  3.43
*******************************************************

### Using Princeple Component Analasis after polynomial with different componant numper and check the results

In [20]:
# using n_componests 2,5,8,10 and checking the results.
pca=PCA(n_components=5)
x_train_pca10 =pca.fit_transform(x_train)
x_test_new_pca10 =pca.transform(x_test)

In [21]:
for i in [2,3,4]:
    print("----------- ", "Poly degree: ",i, " -------------")
    poly2 = PolynomialFeatures(degree=i)
    x_train_poly = poly2.fit_transform(x_train)
    x_test_poly = poly2.transform(x_test)
    
    for n in [2,5,8,10]:
        print("----------- ", "PCA No.",n, " -------------")
        pca=PCA(n_components=2)
        x_train_pca =pca.fit_transform(x_train)
        x_test_pca =pca.transform(x_test)
        
        for name,model in models.items():
            print("----------- ", name, " -------------")
            model.fit(x_train_pca,y_train)
            y_train_pred = model.predict(x_train_pca)
            y_test_pred = model.predict(x_test_pca)
            print("Train Acurracy Score", round(model.score(x_train_pca,y_train),2), "Test Acurracy Score", round(model.score(x_test_pca,y_test),2))
            print("Train MAE Score: ", round(mean_absolute_error(y_train_pred,y_train),2),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
            print("*"*60)

-----------  Poly degree:  2  -------------
-----------  PCA No. 2  -------------
-----------  lr  -------------
Train Acurracy Score 0.2 Test Acurracy Score 0.18
Train MAE Score:  3.7 Test MAE Score:  3.74
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.39 Test Acurracy Score 0.08
Train MAE Score:  3.17 Test MAE Score:  3.91
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.85 Test Acurracy Score 0.06
Train MAE Score:  1.52 Test MAE Score:  3.97
************************************************************
-----------  Adboost  -------------
Train Acurracy Score 0.14 Test Acurracy Score 0.13
Train MAE Score:  4.1 Test MAE Score:  4.13
************************************************************
-----------  PCA No. 5  -------------
-----------  lr  -------------
Train Acurracy Score 0.2 Test Acurracy Score 0.18
Train MAE Score:  3.7 Test MAE Score:  3.74


Train Acurracy Score 0.39 Test Acurracy Score 0.08
Train MAE Score:  3.17 Test MAE Score:  3.91
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.85 Test Acurracy Score 0.06
Train MAE Score:  1.52 Test MAE Score:  3.96
************************************************************
-----------  Adboost  -------------
Train Acurracy Score 0.15 Test Acurracy Score 0.13
Train MAE Score:  4.08 Test MAE Score:  4.12
************************************************************
-----------  PCA No. 10  -------------
-----------  lr  -------------
Train Acurracy Score 0.2 Test Acurracy Score 0.18
Train MAE Score:  3.7 Test MAE Score:  3.74
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.39 Test Acurracy Score 0.08
Train MAE Score:  3.17 Test MAE Score:  3.91
************************************************************
-----------  rf  -------------
Train Acurracy Sc

### As all of the models shows bad results and befor going back to preprocessing a gain I will try to deal the model as classification model with target class between 1 to 20 days as labels and check the results.

In [22]:
# First we need to change the target to numerical to string using dictionery and map it to y train & test
str_target = {1:'1',2:'2',3:'3',4:'4',5:'5',6:'6',7:'7',8:'8',9:'9',10:'10',11:'11',12:'12',13:'13',14:'14',15:'15',16:'16',17:'17',18:'18',19:'19',20:'20'}
y_train = y_train.map(str_target)
y_test = y_test.map(str_target)

In [23]:
y_train.value_counts().sort_values()

19     117
18     134
17     145
16     189
15     216
13     290
14     303
12     357
11     414
10     488
9      591
8      721
20     859
7      892
6     1054
5     1256
4     1471
1     1590
3     1759
2     1814
Name: length_of_stay, dtype: int64

In [24]:
# trying three different classification models to compare between them
models = {"lg": LogisticRegression(),
          "knnc": KNeighborsClassifier(),
          "rfc": RandomForestClassifier(),
         "Adaboost_C": AdaBoostClassifier(),
         "xgbc": xgb.XGBClassifier()}

In [25]:
for name,model in models.items():
        print("----------- ", name, " -------------")
        model.fit(x_train_scaled,y_train)
        y_train_pred = model.predict(x_train_scaled)
        y_test_pred = model.predict(x_test_scaled)
        print("Train Acurracy Score", round(model.score(x_train_scaled,y_train),2), "Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
        print("Train Precision Score: ", round(precision_score(y_train,y_train_pred,average='micro'),2),"Test Precision Score: ", round(precision_score(y_test,y_test_pred,average='micro'),2))
        print("Train Recall Score: ", round(recall_score(y_train,y_train_pred,average='micro'),2),"Test Recall Score: ", round(recall_score(y_test,y_test_pred,average='micro'),2))
        print("*"*60)

-----------  lg  -------------
Train Acurracy Score 0.17 Test Acurracy Score 0.16
Train Precision Score:  0.17 Test Precision Score:  0.16
Train Recall Score:  0.17 Test Recall Score:  0.16
************************************************************
-----------  knnc  -------------
Train Acurracy Score 0.36 Test Acurracy Score 0.13
Train Precision Score:  0.36 Test Precision Score:  0.13
Train Recall Score:  0.36 Test Recall Score:  0.13
************************************************************
-----------  rfc  -------------
Train Acurracy Score 0.96 Test Acurracy Score 0.14
Train Precision Score:  0.96 Test Precision Score:  0.14
Train Recall Score:  0.96 Test Recall Score:  0.14
************************************************************
-----------  Adaboost_C  -------------
Train Acurracy Score 0.17 Test Acurracy Score 0.15
Train Precision Score:  0.17 Test Precision Score:  0.15
Train Recall Score:  0.17 Test Recall Score:  0.15
**********************************************

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], got ['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '3' '4'
 '5' '6' '7' '8' '9']

In [27]:
# tring Cross validation with random forest classifier
rfc = RandomForestClassifier()
cv = cross_validate(rfc,x_train,y_train,cv=10,return_train_score=True)

In [29]:
cv['test_score'].mean()

0.13287858117326057

In [33]:
print(cv)

{'fit_time': array([2.19315076, 2.15823007, 2.16222811, 2.15523648, 2.14023948,
       2.10935903, 2.12531281, 2.13329577, 2.16620278, 2.12431645]), 'score_time': array([0.06980038, 0.07080817, 0.07080078, 0.07280636, 0.07081223,
       0.0698123 , 0.07180882, 0.07480001, 0.07280684, 0.07081032]), 'test_score': array([0.12482947, 0.13710778, 0.12960437, 0.13369714, 0.13301501,
       0.12210095, 0.14802183, 0.13233288, 0.1377899 , 0.13028649]), 'train_score': array([0.96801576, 0.96786418, 0.96756101, 0.9667273 , 0.96847052,
       0.96718205, 0.96589359, 0.96566621, 0.96657572, 0.96559042])}


In [34]:
# parameters for Random Forest model
params = {'criterion':['gini', 'entropy'], 'n_estimators':range(20, 300)}

In [36]:
# using the grid search model for tuning
grid_search = GridSearchCV(estimator=rfc,
                          param_grid=params,
                          scoring="accuracy",
                          n_jobs=-1)

grid_search.fit(x_train_scaled,y_train)

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': range(20, 300)},
             scoring='accuracy')

In [37]:
grid_search.best_score_

0.13635743519781718

In [38]:
grid_search.best_params_

{'criterion': 'gini', 'n_estimators': 88}

* The data is highly imbalance that could explain the bad model accuracy.

In [1]:
# using random under sampler to maintain inbalance of the target class
# resampler = RandomUnderSampler(sampling_strategy='auto')
resampler = SMOTE(sampling_strategy='auto')

NameError: name 'SMOTE' is not defined

In [ ]:
x_train_resample,y_train_resample = resampler.fit_resample(x_train,y_train)

In [ ]:
y_train_resample.value_counts().sort_values()

* after dealing with inbalanced classes now we can check the results of classification

In [ ]:
y_train_resample = pd.DataFrame((y_train_resample),columns=['length_of_stay'])
y_train_resample = np.ravel(y_train_resample)

In [ ]:
# Using Stander Scaler for x_train_resampled, x_test
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_resample)
x_test_scaled = scaler.transform(x_test)

In [ ]:
for name,model in models.items():
        print("----------- ", name, " -------------")
        model.fit(x_train_scaled,y_train_resample)
        y_train_pred = model.predict(x_train_resample)
        y_test_pred = model.predict(x_test_scaled)
        print("Train Acurracy Score", round(model.score(x_train_scaled,y_train_resample),2), "Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
        print("Train Precision Score: ", round(precision_score(y_train_resample,y_train_pred,average='micro'),2),"Test Precision Score: ", round(precision_score(y_test,y_test_pred,average='micro'),2))
        print("Train Recall Score: ", round(recall_score(y_train_resample,y_train_pred,average='micro'),2),"Test Recall Score: ", round(recall_score(y_test,y_test_pred,average='micro'),2))
        print("*"*60)

In [ ]:
ad_b_C = AdaBoostClassifier()
ad_b_C.fit(x_train_scaled,y_train_resample)
y_train_pred = model.predict(x_train_resample)
y_test_pred = model.predict(x_test_scaled)
print("Train Acurracy Score", round(model.score(x_train_scaled,y_train_resample),2), "Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
print("Train Precision Score: ", round(precision_score(y_train_resample,y_train_pred,average='micro'),2),"Test Precision Score: ", round(precision_score(y_test,y_test_pred,average='micro'),2))
print("Train Recall Score: ", round(recall_score(y_train_resample,y_train_pred,average='micro'),2),"Test Recall Score: ", round(recall_score(y_test,y_test_pred,average='micro'),2))
print("*"*60)

In [ ]:
# Random forest shows increase the accuracy score to 81% but it is highly over fitted
rfc = RandomForestClassifier(n_estimators=300,max_depth=20,max_leaf_nodes=20)
rfc.fit(x_train_scaled,y_train_resample)
y_train_pred = model.predict(x_train_resample)
y_test_pred = model.predict(x_test_scaled)
print("Train Acurracy Score", round(model.score(x_train_scaled,y_train_resample),2), "Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
print("Train Precision Score: ", round(precision_score(y_train_resample,y_train_pred,average='micro'),2),"Test Precision Score: ", round(precision_score(y_test,y_test_pred,average='micro'),2))
print("Train Recall Score: ", round(recall_score(y_train_resample,y_train_pred,average='micro'),2),"Test Recall Score: ", round(recall_score(y_test,y_test_pred,average='micro'),2))
print("*"*60)

In [ ]:
feat_imp = list(rfc.feature_importances_)
col_name = list(x_train_resample.columns)
feat_imp_df = pd.DataFrame({"col_name":col_name,"feat_imp":feat_imp})

In [ ]:
feat_imp_df[['col_name','feat_imp']].sort_values(by='feat_imp')

In [ ]:
# Using Stander Scaler for x_train_resampled, x_test
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_resample)
x_test_scaled = scaler.transform(x_test)

In [ ]:
# Random forest shows increase the accuracy score to 81% but it is highly over fitted
rfc = RandomForestClassifier(n_estimators=300)
rfc.fit(x_train_scaled,y_train_resample)
y_train_pred = model.predict(x_train_resample)
y_test_pred = model.predict(x_test_scaled)
print("Train Acurracy Score", round(model.score(x_train_scaled,y_train_resample),2), "Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
print("Train Precision Score: ", round(precision_score(y_train_resample,y_train_pred,average='micro'),2),"Test Precision Score: ", round(precision_score(y_test,y_test_pred,average='micro'),2))
print("Train Recall Score: ", round(recall_score(y_train_resample,y_train_pred,average='micro'),2),"Test Recall Score: ", round(recall_score(y_test,y_test_pred,average='micro'),2))
print("*"*60)

In [ ]:
ad_b_C = AdaBoostClassifier()
ad_b_C.fit(x_train_scaled,y_train_resample)
y_train_pred = model.predict(x_train_resample)
y_test_pred = model.predict(x_test_scaled)
print("Train Acurracy Score", round(model.score(x_train_scaled,y_train_resample),2), "Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
print("Train Precision Score: ", round(precision_score(y_train_resample,y_train_pred,average='micro'),2),"Test Precision Score: ", round(precision_score(y_test,y_test_pred,average='micro'),2))
print("Train Recall Score: ", round(recall_score(y_train_resample,y_train_pred,average='micro'),2),"Test Recall Score: ", round(recall_score(y_test,y_test_pred,average='micro'),2))
print("*"*60)

In [ ]:
xgbc = xgb.XGBClassifier()
xgbc.fit(x_train_scaled,y_train_resample)
y_train_pred = model.predict(x_train_resample)
y_test_pred = model.predict(x_test_scaled)
print("Train Acurracy Score", round(model.score(x_train_scaled,y_train_resample),2), "Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
print("Train Precision Score: ", round(precision_score(y_train_resample,y_train_pred,average='micro'),2),"Test Precision Score: ", round(precision_score(y_test,y_test_pred,average='micro'),2))
print("Train Recall Score: ", round(recall_score(y_train_resample,y_train_pred,average='micro'),2),"Test Recall Score: ", round(recall_score(y_test,y_test_pred,average='micro'),2))
print("*"*60)

### The Models evaluations results above indicate that there are both under fitting and over fitting in the same time which mean we have to go back again to the preprocessing steps and try to deal with that for example by dropping outliers and also try different way of Categorical incoding like binary encoding.